In [1]:
# Initial imports.
import pandas as pd
import numpy as np
from path import Path
from ast import literal_eval
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
import csv
import random

In [2]:
# Reading the clean data
file = 'fully_encoded_recipes_with_cuisine_from_SP.csv'
recipe_df = pd.read_csv(file, index_col=0)
recipe_df

,cuisine_SP,onion,garlic,vanilla,lemon,bell pepper,tomato,chocolate,mushrooms,italian cheese,...,harissa,sprouts,squash blossoms,grapefruit,tamarind pulp,savory,baharat,douchi,sucralose,jicama
5,8,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10,11,0,1,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16,2,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20,10,1,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9978,11,1,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
9981,10,1,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
9984,10,0,1,0,0,1,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
9988,11,1,1,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
encoding_key = pd.read_csv('cuisine_encoding_dict.csv', index_col=0)
encoding_key

,African,American,Asian,Creole,English,European,French,Greek,Indian,Irish,Italian,Mexican,Middle Eastern,Southern,Spanish
encoded,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14


## Data are processed to dataframes with and without response variables (cuisines) and the response variables are encoded to variables

In [4]:
greek_df = recipe_df[recipe_df.cuisine_SP==7].copy()
non_greek_df = recipe_df[recipe_df.cuisine_SP!=7]
non_greek_df_indexes = pd.DataFrame(non_greek_df.index.values, columns=['recipe_num'])
non_greek_df_indexes

,recipe_num
0,5
1,10
2,16
3,17
4,20
...,...
2878,9978
2879,9981
2880,9984
2881,9988


In [5]:
index_for_balance = set()
while len(index_for_balance)<=len(greek_df):
    index_for_balance.add(random.randint(0,2882))
index_for_balance = list(index_for_balance)
non_greek_df_balance = non_greek_df.iloc[index_for_balance]
non_greek_df_balance
    

,cuisine_SP,onion,garlic,vanilla,lemon,bell pepper,tomato,chocolate,mushrooms,italian cheese,...,harissa,sprouts,squash blossoms,grapefruit,tamarind pulp,savory,baharat,douchi,sucralose,jicama
6276,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4139,6,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7542,11,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3676,2,1,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
6278,10,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7073,1,1,0,0,0,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
441,11,1,1,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
7505,5,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1872,13,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
greek_df_balanced = pd.concat([greek_df, non_greek_df_balance])

In [7]:
greek_df_balanced

,cuisine_SP,onion,garlic,vanilla,lemon,bell pepper,tomato,chocolate,mushrooms,italian cheese,...,harissa,sprouts,squash blossoms,grapefruit,tamarind pulp,savory,baharat,douchi,sucralose,jicama
136,7,1,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
223,7,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
286,7,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
492,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
567,7,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7073,1,1,0,0,0,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
441,11,1,1,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
7505,5,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1872,13,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
greek_df_balanced['response'] = 0
greek_df_balanced.response = greek_df_balanced.response.mask(greek_df_balanced.cuisine_SP==7, 1)
greek_df_balanced.drop(columns='cuisine_SP', inplace=True)

In [9]:
#Define the features set
X = greek_df_balanced.copy()
X = X.drop("response", axis=1)
X

,onion,garlic,vanilla,lemon,bell pepper,tomato,chocolate,mushrooms,italian cheese,parsley,...,harissa,sprouts,squash blossoms,grapefruit,tamarind pulp,savory,baharat,douchi,sucralose,jicama
136,1,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
223,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
286,0,1,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
492,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
567,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7073,1,0,0,0,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
441,1,1,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
7505,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1872,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
# define the target

y = greek_df_balanced["response"]
y

136     1
223     1
286     1
492     1
567     1
       ..
7073    0
441     0
7505    0
1872    0
5373    0
Name: response, Length: 139, dtype: int64

In [11]:
# Splitting into Train and Test sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [12]:
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=100, random_state=78) 

In [13]:
# Fitting the model
rf_model = rf_model.fit(X_train, y_train)

In [14]:
# Making predictions using the testing data.
predictions = rf_model.predict(X_test)


In [15]:
# Calculating the confusion matrix.
cm = confusion_matrix(y_test, predictions)

# Create a DataFrame from the confusion matrix.
italian_cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], 
    columns=["Predicted 0", "Predicted 1"])
    
italian_cm_df

,Predicted 0,Predicted 1
Actual 0,18,1
Actual 1,0,16


In [16]:
importance = rf_model.feature_importances_

In [17]:
importance_df = pd.DataFrame(importance, index=X.columns.values, columns=['feature_importance'])
importance_df = importance_df.sort_values('feature_importance', ascending=False)

In [18]:
importance_df.iloc[0:10,:]

,feature_importance
yogurt,0.137611
eggplant,0.114840
mediterranean cheese,0.088070
lemon,0.054769
cucumber,0.041740
allspice,0.036641
dill,0.034079
puff pastry,0.026048
celery,0.024428
vanilla,0.023889
